In [4]:
from pathlib import Path
import os
import sys
import re
import yaml
# from sklearn.model_selection import train_test_split
import cv2
import random
from collections import defaultdict
from tqdm import tqdm
import math

In [12]:

gang_videos = list(Path(rf"G:\My Drive\dataset\video\trim").rglob("*/normal*.mp4"))
len(gang_videos)

94

In [8]:
image.shape, out.shape

((8, 8), (10, 10))

In [ ]:
cap = cv2.VideoCapture("video.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

writer = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    annotated_frame = yolo.predict(frame)
    writer.write(annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

## video frame == 32],

In [6]:
dir = Path(rf"G:\My Drive\dataset\video\trim")
under30fps = []

for file in dir.glob("*.mp4"):
    cap = cv2.VideoCapture(str(file))
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    if fps < 30:
        print(file, fps)
        under30fps.append(file)
        new_file = file.with_name(file.stem + "_30fps.mp4")
        !ffmpeg -i "{str(file)}" -c:v libx264 -vf "fps=30" "{str(new_file)}" -hide_banner -loglevel error
        file.rename(dir / "sampah" / file.name)
        new_file.rename(file)


G:\My Drive\dataset\video\trim\geng-3A.mp4 25.105838338091957
G:\My Drive\dataset\video\trim\geng-4A.mp4 25.027311550302908
G:\My Drive\dataset\video\trim\geng-7.mp4 25.031654816402064
G:\My Drive\dataset\video\trim\geng-8.mp4 25.107045543012845
G:\My Drive\dataset\video\trim\geng-9.mp4 25.054538242876976
G:\My Drive\dataset\video\trim\geng-11.mp4 25.0
G:\My Drive\dataset\video\trim\geng-11A.mp4 25.004139758238118
G:\My Drive\dataset\video\trim\geng-11B.mp4 25.05604641962284
G:\My Drive\dataset\video\trim\geng-11C.mp4 25.01326963906582
G:\My Drive\dataset\video\trim\geng-11D.mp4 25.023214896632616
G:\My Drive\dataset\video\trim\normal-1.000.mp4 25.0
G:\My Drive\dataset\video\trim\normal-1.001.mp4 24.25
G:\My Drive\dataset\video\trim\normal-1.002.mp4 25.0
G:\My Drive\dataset\video\trim\normal-1.003.mp4 25.0
G:\My Drive\dataset\video\trim\normal-1.004.mp4 25.0
G:\My Drive\dataset\video\trim\normal-1.005.mp4 25.0
G:\My Drive\dataset\video\trim\normal-1.006.mp4 25.0
G:\My Drive\dataset\vid

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'G:\\My Drive\\dataset\\video\\trim\\sampah\\geng-X_30fps.mp4' -> 'G:\\My Drive\\dataset\\video\\trim\\sampah\\geng-X.mp4'

## video data splitter

In [2]:
os.symlink(rf"G:\My Drive\dataset\video\train\geng-4.mp4", rf"G:\My Drive\dataset\video\trim\geng-4.mp4")

OSError: [WinError 1] Incorrect function: 'G:\\My Drive\\dataset\\video\\train\\geng-4.mp4' -> 'G:\\My Drive\\dataset\\video\\trim\\geng-4.mp4'

In [10]:
def list_videos(directory):
    """List all video files in the directory."""
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(('.mp4', '.avi', '.mov'))]

def get_total_frames(video_path):
    """Get the total number of frames in the video."""
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    return total_frames

def calculate_matrices(total_frames):
    """Calculate the total number of 30x30 matrices generated from the video."""
    if total_frames > 145:
        return 5 * ((total_frames - 145) // 5) + 5
    else:
        return 5

def stratified_split(directory, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    """Perform a stratified split on videos based on their class."""
    if train_ratio + val_ratio + test_ratio != 1.0:
        raise ValueError("ngaco")
    # Step 1: List all video files
    video_files = list_videos(directory)
    # Step 2 & 3: Read each video, calculate generated matrices, and group by class
    videos_by_class = defaultdict(list)
    total_matrices = 0

    for video in video_files:
        total_frames = get_total_frames(video)
        n_matrices = calculate_matrices(total_frames)
        total_matrices += n_matrices

        # Determine class label based on filename
        class_label = 'geng' if 'geng' in os.path.basename(video).lower() else 'normal'
        videos_by_class[class_label].append((video, n_matrices))

    train_videos, val_videos, test_videos = [], [], []
    train_matrices, val_matrices, test_matrices = 0, 0, 0
    # matrices_per_class = defaultdict(int)
    stats = dict(train = dict(), val = dict(), test = dict())
    # Iterate over each class
    remainder_videos = []
    for class_label, videos in videos_by_class.items():
        random.shuffle(videos)  # Shuffle videos of each class to randomize splitting

        # Calculate the target matrices for the current class
        total_class_matrices = sum(matrices for _, matrices in videos)
        # print(f"Class: {class_label}, Total videos: {len(videos)}, Total matrices: {total_class_matrices}")
        target_train_matrices_class = math.ceil(train_ratio * total_class_matrices)
        target_val_matrices_class = math.ceil(val_ratio * total_class_matrices)
        target_test_matrices_class = int(test_ratio * total_class_matrices)

        # Split videos for this class
        current_train_matrices, current_val_matrices, current_test_matrices = 0, 0, 0
        current_train_videos, current_val_videos, current_test_videos = [], [], []
        for video_path, n_matrices in videos:
            if current_train_matrices < target_train_matrices_class and \
               (current_train_matrices + n_matrices <= target_train_matrices_class):
                current_train_videos.append(video_path)
                current_train_matrices += n_matrices
            elif current_val_matrices < target_val_matrices_class and \
                 (current_val_matrices + n_matrices <= target_val_matrices_class):
                current_val_videos.append(video_path)
                current_val_matrices += n_matrices
            elif current_test_matrices < target_test_matrices_class and \
                 (current_test_matrices + n_matrices <= target_test_matrices_class):
                current_test_videos.append(video_path)
                current_test_matrices += n_matrices
            else:
                remainder_videos.append(dict(video = video_path, matrices = n_matrices))

        train_videos.extend(current_train_videos)
        val_videos.extend(current_val_videos)
        test_videos.extend(current_test_videos)
        train_matrices += current_train_matrices
        val_matrices += current_val_matrices
        test_matrices += current_test_matrices
        # matrices_per_class[class_label] = (current_train_matrices, current_val_matrices)
        print(f"Class: {class_label}, Train: {current_train_matrices}, Val: {current_val_matrices}, test: {current_test_matrices}")
        stats['train'][class_label] = current_train_matrices
        stats['val'][class_label] = current_val_matrices
        stats['test'][class_label] = current_test_matrices
    
    stats['total'] = dict(train = train_matrices, val = val_matrices, test = test_matrices)
    stats['remainder'] = remainder_videos
    return train_videos, val_videos, test_videos, stats



In [12]:
# Step 4: Perform stratified split
train_videos, val_videos, test_videos, stats = stratified_split(rf"G:\My Drive\dataset\video\trim", 0.7, 0.15, 0.15)

print("Training Set:")
print(f"Total videos: {len(train_videos)}, Total matrices: {stats['total']['train']}")

print("Validation Set:")
print(f"Total videos: {len(val_videos)}, Total matrices: {stats['total']['val']}")

print("Test Set:")
print(f"Total videos: {len(test_videos)}, Total matrices: {stats['total']['test']}")

print("Remainder:")
for rem in stats['remainder']:
    print(rem["video"], rem["matrices"])

train_videos, val_videos, test_videos


Class: geng, Train: 2550, Val: 525, test: 470
Class: normal, Train: 2580, Val: 505, test: 490
Training Set:
Total videos: 78, Total matrices: 5130
Validation Set:
Total videos: 10, Total matrices: 1030
Test Set:
Total videos: 5, Total matrices: 960
Remainder:
G:\My Drive\dataset\video\trim\geng-2A.mp4 110
G:\My Drive\dataset\video\trim\normal-2.006.mp4 110


(['G:\\My Drive\\dataset\\video\\trim\\geng-2.mp4',
  'G:\\My Drive\\dataset\\video\\trim\\geng-10.mp4',
  'G:\\My Drive\\dataset\\video\\trim\\geng-7.mp4',
  'G:\\My Drive\\dataset\\video\\trim\\geng-11.mp4',
  'G:\\My Drive\\dataset\\video\\trim\\geng-11B.mp4',
  'G:\\My Drive\\dataset\\video\\trim\\geng-5-x264.mp4',
  'G:\\My Drive\\dataset\\video\\trim\\geng-1.mp4',
  'G:\\My Drive\\dataset\\video\\trim\\geng-10_30fps.mp4',
  'G:\\My Drive\\dataset\\video\\trim\\geng-11C.mp4',
  'G:\\My Drive\\dataset\\video\\trim\\geng-3A.mp4',
  'G:\\My Drive\\dataset\\video\\trim\\geng-4.mp4',
  'G:\\My Drive\\dataset\\video\\trim\\geng-8.mp4',
  'G:\\My Drive\\dataset\\video\\trim\\geng-p4-ff.mp4',
  'G:\\My Drive\\dataset\\video\\trim\\geng-9.mp4',
  'G:\\My Drive\\dataset\\video\\trim\\geng-11A.mp4',
  'G:\\My Drive\\dataset\\video\\trim\\geng-6.mp4',
  'G:\\My Drive\\dataset\\video\\trim\\normal-3.003.mp4',
  'G:\\My Drive\\dataset\\video\\trim\\normal-1.000.mp4',
  'G:\\My Drive\\dataset\\v

In [14]:
# move remainder to train
train_videos.extend([rem["video"] for rem in stats['remainder']])
stats['total']['train'] += sum(rem["matrices"] for rem in stats['remainder'])

In [17]:
for rem in stats['remainder']:
    if "geng" in rem["video"]:
        stats['train']['geng'] += rem["matrices"]
    else:
        stats['train']['normal'] += rem["matrices"]

In [20]:
stats["remainder"].clear()
stats



{'train': {'geng': 2660, 'normal': 2690},
 'val': {'geng': 525, 'normal': 505},
 'test': {'geng': 470, 'normal': 490},
 'total': {'train': 5350, 'val': 1030, 'test': 960},
 'remainder': []}

In [21]:

def save_split_set(set_name, dest_dir, train_set, val_set, test_set, stats=None):
    split_set = dict(train=train_set, val=val_set, test=test_set)
    if stats:
        split_set['stats'] = stats
    
    # move the videos to the respective folders, train and val, make directories if not exist
    train_dir = dest_dir / "train"
    val_dir = dest_dir / "val"
    test_dir = dest_dir / "test"
    set_path_file = (dest_dir / set_name)
    print(f"saving split set to {set_path_file}")
    with set_path_file.open('w') as f:
            yaml.dump(split_set, f)

    train_dir.mkdir(parents=True, exist_ok=True)
    val_dir.mkdir(parents=True, exist_ok=True)
    test_dir.mkdir(parents=True, exist_ok=True)

    print("Moving videos to train folders")
    for video in tqdm(train_set):
        video_name = Path(video).name
        os.rename(video, train_dir / video_name)

    print("Moving videos to val folders")
    for video in tqdm(val_set):
        video_name = Path(video).name
        os.rename(video, val_dir / video_name)
    
    print("Moving videos to test folders")
    for video in tqdm(test_set):
        video_name = Path(video).name
        os.rename(video, test_dir / video_name)


dest_dir = Path(rf"G:\My Drive\dataset\video\trim")
save_split_set("set-D.yaml", dest_dir,
               train_videos, val_videos, test_videos, stats)

saving split set to G:\My Drive\dataset\video\trim\set-D.yaml
Moving videos to train folders


100%|██████████| 80/80 [00:00<00:00, 109.90it/s]


Moving videos to val folders


100%|██████████| 10/10 [00:00<00:00, 137.25it/s]


Moving videos to test folders


100%|██████████| 5/5 [00:00<00:00, 129.98it/s]


In [5]:
def revert_move(dest_dir):
    train_dir = dest_dir / "train"
    val_dir = dest_dir / "val"
    test_dir = dest_dir / "test"

    for video in tqdm(train_dir.glob("*.mp4")):
        video_name = video.name
        os.rename(video, dest_dir / video_name)

    for video in tqdm(val_dir.glob("*.mp4")):
        video_name = video.name
        os.rename(video, dest_dir / video_name)
    
    for video in tqdm(test_dir.glob("*.mp4")):
        video_name = video.name
        os.rename(video, dest_dir / video_name)


train_dir = Path(rf"G:\My Drive\dataset\video\train")
val_dir = Path(rf"G:\My Drive\dataset\video\val")
test_dir = Path(rf"G:\My Drive\dataset\video\test")
dest_dir = Path(rf"G:\My Drive\dataset\video\trim")
revert_move(dest_dir)

77it [00:00, 79.75it/s]
10it [00:00, 80.23it/s]
6it [00:00, 108.74it/s]


In [37]:
xy = [(5, 0), (3, 1), (2, 1)]
x, y = zip(*xy)
x, y 

((5, 3, 2), (0, 1, 1))

## stuff

In [2]:
dataset_dir = Path(rf"G:\My Drive\dataset\all-data-fix")

images_path = []
labels_path = []
missing_labels = 0

for img_path in (dataset_dir / "images").glob("*.jpg"):
    images_path.append(img_path)
    label_path = (dataset_dir / "labels" / img_path.stem).with_suffix(".txt")
    if label_path.exists():
        labels_path.append(label_path)
    else:
        print(f"Missing label for {img_path}")
        labels_path.append(None)
        missing_labels += 1

print(f"Missing labels: {missing_labels}")
print(f"Total images: {len(images_path)}")

Missing labels: 0
Total images: 1290


In [3]:
train_images, val_images, train_annotations, val_annotations = train_test_split(
    images_path, labels_path, test_size=0.2, random_state=42
)

# Optionally, split validation set further into validation and test sets (80-10-10 split)
val_images, test_images, val_annotations, test_annotations = train_test_split(
    val_images, val_annotations, test_size=0.5, random_state=42
)

print(f"Train images: {len(train_images)}")
print(f"Val images: {len(val_images)}")
print(f"Test images: {len(test_images)}")

Train images: 1032
Val images: 129
Test images: 129


In [6]:
def move_files(images, annotations, target_dir):
    target_dir.mkdir(parents=True, exist_ok=True)
    (target_dir / "images").mkdir(parents=True, exist_ok=True)
    (target_dir / "labels").mkdir(parents=True, exist_ok=True)
    
    for img_path, label_path in zip(images, annotations):
        img_path.rename(target_dir / "images" / img_path.name)
        if label_path:
            label_path.rename(target_dir / "labels" / label_path.name)

In [7]:
print("Moving train set...")
move_files(train_images, train_annotations, dataset_dir / "train")
print("Moving val set...")
move_files(val_images, val_annotations, dataset_dir / "val")
print("Moving test set...")
move_files(test_images, test_annotations, dataset_dir / "test")
print("Done!")

Moving train set...
Moving val set...
Moving test set...
Done!


In [4]:
conf = yaml.safe_load(open("all-data.yaml"))
conf

{'train': ['./normal-1/train',
  './normal-2/train',
  './normal-3/train',
  './normal-4/train',
  './normal-5/train',
  './geng-1/train',
  './geng-2/train',
  './geng-3/train',
  './geng-pri-1/train',
  './geng-pri-2/train',
  './geng-pri-3/train',
  './geng-pri-4/train',
  './geng-pri-5/train',
  './geng-pri-6/train'],
 'val': ['./normal-1/val',
  './normal-2/val',
  './normal-3/val',
  './normal-4/val',
  './normal-5/val',
  './geng-1/val',
  './geng-2/val',
  './geng-3/val',
  './geng-pri-1/val',
  './geng-pri-2/val',
  './geng-pri-3/val',
  './geng-pri-4/val',
  './geng-pri-5/val',
  './geng-pri-6/val'],
 'nc': 1,
 'names': ['motor']}

In [5]:
DESTINATION_PATH = Path("all-data")

In [43]:
def move_content(dir):
    for img_path in (Path(dir) / "images").glob("*.jpg"):
    # print(img)
        img_name = img_path.stem
        source_data = img_path.parent.parent.parent.name

        label_path = img_path.parent.parent / "labels" / f"{img_name}.txt"

        img_dest_dir = (DESTINATION_PATH / "images")
        img_dest_dir.mkdir(parents=True, exist_ok=True)
        label_dest_dir = (DESTINATION_PATH / "labels")
        label_dest_dir.mkdir(parents=True, exist_ok=True)

        img_dest = img_dest_dir / f"{source_data}--{img_name}.jpg"
        label_dest = label_dest_dir / f"{source_data}--{img_name}.txt"

        # check duplicate before moving
        if img_dest.exists() and label_dest.exists():
            print(f"Duplicate {img_dest} and {label_dest}")
            continue

        print(f"moving {img_path} to {img_dest}")
        print(f"moving {label_path} to {label_dest}")
        img_path.rename(img_dest)
        if label_path.exists():
            label_path.rename(label_dest)
        else:
            print(f"Label not found for {img_path}, skipping...")
        print()

In [44]:
for dir in conf["train"] + conf["val"]:
    print("="*20)
    print(f"moving content from {dir}")
    move_content(dir)

moving content from ./normal-1/train
moving content from ./normal-2/train
moving content from ./normal-3/train
moving content from ./normal-4/train
moving normal-4\train\images\frame-n4-034.jpg to all-data\images\normal-4--frame-n4-034.jpg
moving normal-4\train\labels\frame-n4-034.txt to all-data\labels\normal-4--frame-n4-034.txt
Label not found for normal-4\train\images\frame-n4-034.jpg, skipping...

moving normal-4\train\images\frame-n4-035.jpg to all-data\images\normal-4--frame-n4-035.jpg
moving normal-4\train\labels\frame-n4-035.txt to all-data\labels\normal-4--frame-n4-035.txt
Label not found for normal-4\train\images\frame-n4-035.jpg, skipping...

moving normal-4\train\images\frame-n4-036.jpg to all-data\images\normal-4--frame-n4-036.jpg
moving normal-4\train\labels\frame-n4-036.txt to all-data\labels\normal-4--frame-n4-036.txt

moving normal-4\train\images\frame-n4-037.jpg to all-data\images\normal-4--frame-n4-037.jpg
moving normal-4\train\labels\frame-n4-037.txt to all-data\lab

In [10]:
# visualize data
import cv2
import time


def draw_bounding_box(img, label_path):
    try:
        with open(label_path, "r") as f:
            for line in f:
                x, y, w, h = map(float, line.split()[1:])
                x1, y1 = int((x - w/2) * img.shape[1]), int((y - h/2) * img.shape[0])
                x2, y2 = int((x + w/2) * img.shape[1]), int((y + h/2) * img.shape[0])
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    except FileNotFoundError:
        print(f"Label file not found: {label_path}")
    cv2.putText(img, label_path.stem, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    return img

files = list(DESTINATION_PATH.glob("images/*.jpg"))

In [12]:
for img_path in files:
    img = cv2.imread(str(img_path))
    label_path = DESTINATION_PATH / "labels" / f"{img_path.stem}.txt"
    img = draw_bounding_box(img, label_path)
    # scale images to fit screen
    scale_percent = 50
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    
    cv2.imshow("img", img)
    if cv2.waitKey(100) == ord("q"):
        break

cv2.destroyAllWindows()

Label file not found: all-data\labels\normal-4--frame-n4-033.txt
Label file not found: all-data\labels\normal-4--frame-n4-034.txt
Label file not found: all-data\labels\normal-4--frame-n4-035.txt
Label file not found: all-data\labels\normal-4--frame-n4-039.txt
Label file not found: all-data\labels\geng-pri-1--img-001.txt
Label file not found: all-data\labels\geng-pri-3--img-001.txt
Label file not found: all-data\labels\geng-pri-3--img-002.txt
Label file not found: all-data\labels\geng-pri-3--img-003.txt
Label file not found: all-data\labels\geng-pri-3--img-004.txt
Label file not found: all-data\labels\geng-pri-3--img-005.txt
Label file not found: all-data\labels\geng-pri-3--img-006.txt
Label file not found: all-data\labels\geng-pri-3--img-007.txt
Label file not found: all-data\labels\geng-pri-3--img-008.txt
Label file not found: all-data\labels\geng-pri-3--img-009.txt
Label file not found: all-data\labels\geng-pri-3--img-010.txt
Label file not found: all-data\labels\geng-pri-3--img-011.

In [19]:

cv2.destroyAllWindows()